## Import packages

In [1]:
from abc import ABC, abstractmethod
from typing import List
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModelForCausalLM, ViltProcessor, ViltForQuestionAnswering
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from models.vilt import VILTModel
from models.git import GitModel
from trainer.singlemodeltrainer import Trainer
from trainer.jointmodeltrainer import JointTrainer

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [10]:
url_eiffel = "https://www.travelandleisure.com/thmb/SPUPzO88ZXq6P4Sm4mC5Xuinoik=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/eiffel-tower-paris-france-EIFFEL0217-6ccc3553e98946f18c893018d5b42bde.jpg"
url_obama = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/President_Barack_Obama.jpg/440px-President_Barack_Obama.jpg"
url_cat = "http://images.cocodataset.org/val2017/000000039769.jpg"
url_lion = "https://upload.wikimedia.org/wikipedia/commons/thumb/7/73/Lion_waiting_in_Namibia.jpg/1200px-Lion_waiting_in_Namibia.jpg"
url_dog = "https://www.princeton.edu/sites/default/files/styles/1x_full_2x_half_crop/public/images/2022/02/KOA_Nassau_2697x1517.jpg?"

In [11]:
image = Image.open(requests.get(url_obama, stream=True).raw)
image1 = Image.open(requests.get(url_cat, stream=True).raw)
image2 = Image.open(requests.get(url_eiffel, stream=True).raw)
image_dog = Image.open(requests.get(url_dog, stream=True).raw)

In [12]:
b = GitModel()
b.target = "cat"

In [13]:
a = VILTModel()
a.target = "cat"

In [14]:
t1 = Trainer(model = a, images=[image_dog], texts=["Wat animal is in the image?"],  epochs=100)

In [15]:
t2 = Trainer(model = b, images=[image_dog], texts=["Wat animal is in the image?"], epochs=500)

In [16]:
t3 = JointTrainer(models=[b, a], images=[image_dog], texts=['Wat animal is in the image?'], epochs=50)

In [17]:
from tqdm.auto import tqdm

res_joint = t1.train()
change, img = res_joint

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
from tqdm.auto import tqdm

res_joint = t2.train()
change, img = res_joint

<ipython-input-6-924d98bb0882>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).unsqueeze(0).to(device)


  0%|          | 0/500 [00:00<?, ?it/s]

<ipython-input-6-924d98bb0882>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).unsqueeze(0).to(device)


In [19]:
change.min()

tensor(-0.0610, device='cuda:0', grad_fn=<MinBackward1>)

In [21]:
a.generate_answer('Wat animal is in the image?', img)

'Wat animal is in the image?yes'

In [22]:
res_git = t1.train()
change, changed_image = res_git

  0%|          | 0/100 [00:00<?, ?it/s]

In [23]:
res_vilt = t2.train()
change, changed_image = res_vilt

<ipython-input-6-924d98bb0882>:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).unsqueeze(0).to(device)


  0%|          | 0/500 [00:00<?, ?it/s]

<ipython-input-6-924d98bb0882>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).unsqueeze(0).to(device)


In [ ]:
plt.imshow(img[0].detach().cpu().numpy().transpose(1, 2, 0))

In [ ]:
a.generate_answer("What animal is in the image?", changed_image)

## Load MS-COCO dataset for training adversaries

In [ ]:
dataset = load_dataset("RIW/small-coco", split="train[100:200]")

In [ ]:
t = Trainer(
    model = a,
    images=[image1],
    texts=["What is the color of the sky?", "Is green the same as black?"], epochs=200)

## Train adversaries with ViLT + GIT models

In [ ]:
num_epochs = 50
delta = 0.05
# Sample 20 random indices
# sampled_indices = random.sample(range(len(dataset['train'])), num_image_samples)
# Extract the sampled points from the dataset
# sampled_data = [dataset['train'][index] for index in sampled_indices]

adversaries = []
deltas = []
from tqdm.auto import tqdm
for sample in tqdm(dataset):
  print(sample)
  t = JointTrainer(
      models=[b, a],
      images=[sample['image']],
      texts=['Wat animal is in the image?'],
      epochs=num_epochs,
      delta = delta
    )
  change, image = t.train()
  deltas.append(change)
  adversaries.append(image)

## Train adversaries with GIT model

In [ ]:
num_epochs = 50
delta = 0.05
adversaries_git = []
deltas_git = []
from tqdm.auto import tqdm
for sample in tqdm(dataset):
  print(sample)
  t = Trainer(
      model=b,
      images=[sample['image']],
      texts=['Wat animal is in the image?'],
      epochs=num_epochs,
      delta = delta
    )
  change, image = t.train()
  deltas_git.append(change)
  adversaries_git.append(image)

## Train adversaries with ViLT model

In [ ]:
num_epochs = 50
delta = 0.05
adversaries_vilt = []
deltas_vilt = []
from tqdm.auto import tqdm
for sample in tqdm(dataset):
  print(sample)
  t = Trainer(
      model=a,
      images=[sample['image']],
      texts=['Wat animal is in the image?'],
      epochs=num_epochs,
      delta = delta
    )
  change, image = t.train()
  deltas_vilt.append(change)
  adversaries_vilt.append(image)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
torch.save(adversaries,'/content/drive/MyDrive/capstone/adversaries.pt')
torch.save(deltas,'/content/drive/MyDrive/capstone/delta.pt')

In [ ]:
torch.save(adversaries_git,'/content/drive/MyDrive/capstone/git/adversaries_git.pt')
torch.save(deltas_git,'/content/drive/MyDrive/capstone/git/delta_git.pt')

In [ ]:
torch.save(adversaries_vilt,'/content/drive/MyDrive/capstone/vilt/adversaries_vilt.pt')
torch.save(deltas_vilt,'/content/drive/MyDrive/capstone/vilt/delta_vilt.pt')

In [ ]:
b.generate_answer('Wat animal is in the image?',)

In [ ]:
for x in deltas_vilt:
  print(x.min())

## L2 Norm of adversaries

In [ ]:
import matplotlib.pyplot as plt
delta_norm = [torch.norm(delta1).detach().cpu().numpy() for delta1 in deltas]
delta_norm_git = [torch.norm(delta1).detach().cpu().numpy() for delta1 in deltas_git]
delta_norm_vilt = [torch.norm(delta1).detach().cpu().numpy() for delta1 in deltas_vilt]

In [ ]:
len(delta_norm_vilt)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.hist(delta_norm,  label='Joint')
ax.hist(delta_norm_git, label='GIT')
ax.hist(delta_norm_vilt,  label='VILT')

ax.set_xlabel('l2 norm adversarial perturbation')
ax.set_ylabel('density')
plt.title('Histogram of l2 norm of adversarial perturbations')
plt.legend()
plt.savefig('histogram.png')

In [ ]:
dummy_img = a.preprocess_image(image1)

## Visualizing embeddings of adversary and original images in visual encoder embedding space

In [ ]:
def plot_pca(adversaries, deltas, name):
  embeddings_adversaries = []
  embeddings_images = []
  for image in adversaries:
    embedding, *rest = a._model.vilt.embeddings.visual_embed(
      image,
      pixel_mask=torch.ones((1, a._model.vilt.config.image_size, a._model.vilt.config.image_size), device=device)
  )
    embeddings_adversaries.append(embedding.mean(dim=1).squeeze().detach().cpu().numpy())

  for i, image in enumerate(adversaries):
    embedding, *rest = a._model.vilt.embeddings.visual_embed(
      image - deltas[i],
      pixel_mask=torch.ones((1, a._model.vilt.config.image_size, a._model.vilt.config.image_size), device=device)
  )
    embeddings_images.append(embedding.mean(dim=1).squeeze().detach().cpu().numpy())



  # Generate random high-dimensional data (50 points, each with 10 features)
  data = np.array(embeddings_images)

  # Perform PCA
  pca = PCA(n_components=2)
  pca_result = pca.fit_transform(data)
  pca_adv = pca_result
  pca_img = pca.transform(embeddings_adversaries)

  # Plot explained variance ratio
  explained_variance_ratio = 100* pca.explained_variance_ratio_
  size = 10
  plt.scatter(pca_img[:, 0], pca_img[:, 1], color='green', s=size, label='Original')
  plt.scatter(pca_adv[:, 0], pca_adv[:, 1], color='red', s=size, label='Adversaries')
  plt.xlabel(f'PCA1 / Variance explained: {explained_variance_ratio[0]:.2f}%')
  plt.ylabel(f'PCA2 / Variance explained: {explained_variance_ratio[1]:.2f}%')
  plt.legend()
  plt.title(f'Visual Embeddings of adversaries trained on {name}')
  plt.tight_layout()
  plt.savefig(f'{name}_embedding.png')
  plt.show();


In [ ]:
plot_pca(adversaries_vilt, deltas_vilt, 'VILT')

In [ ]:
embeddings_adv = []
for image in adversaries_joint:
  embedding, *rest = a._model.vilt.embeddings.visual_embed(
    image,
    pixel_mask=torch.ones((1, a._model.vilt.config.image_size, a._model.vilt.config.image_size), device=device)
)
  embeddings_adv.append(embedding.mean(dim=1).squeeze().detach().cpu().numpy())

embeddings_img = []
for i, image in enumerate(adversaries):
  embedding, *rest = a._model.vilt.embeddings.visual_embed(
    image - deltas[i],
    pixel_mask=torch.ones((1, a._model.vilt.config.image_size, a._model.vilt.config.image_size), device=device)
)
  embeddings_img.append(embedding.mean(dim=1).squeeze().detach().cpu().numpy())

In [ ]:


# Generate random high-dimensional data (50 points, each with 10 features)

data = np.array(embeddings_adv + embeddings_img)

# Perform PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data)
pca_adv = pca_result[:len(embeddings_adv)]
pca_img = pca_result[len(embeddings_adv):]

# Plot explained variance ratio
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
x_values = np.arange(1, len(cumulative_variance_ratio) + 1)

plt.scatter(pca_adv[:, 0], pca_adv[:, 1])
plt.scatter(pca_img[:, 0], pca_img[:, 1], color='red')

In [ ]:
fig, ax = plt.subplots(1, 2)

for weight_decay in [1e-5, 1e-4, 1e-3]:
  a_loss = [_[0] for _ in loss_trajectories[weight_decay]]
  b_loss = [_[1] for _ in loss_trajectories[weight_decay]]
  ax[0].plot(a_loss, label=f'$\lambda = $ {weight_decay}')
  ax[1].plot(b_loss, label=f'$\lambda = $ {weight_decay}')

ax[0].legend()
ax[1].legend()